In [10]:
import cv2
import time

cam = cv2.VideoCapture(0)

cv2.namedWindow("test")

img_counter = 0

while True:
    ret, frame = cam.read()
    
    if frame:
        cv2.imshow("test", frame)
        if not ret:
            break
        k = cv2.waitKey(1)

        if k%256 == 27:
            # ESC pressed
            print("Escape hit, closing...")
            break
        elif k%256 == 32:
            # SPACE pressed
            img_name = "opencv_frame_{}.png".format(img_counter)
            cv2.imwrite(img_name, frame)
            print("{} written!".format(img_name))
            img_counter += 1  
            # find prediction 
            img = image.load_img(img_name, target_size=(250, 250))
            x = image.img_to_array(img)
            x = np.expand_dims(x, axis=0)
            x = preprocess_input(x)
            preds = transfer_model.predict(x)
            pred_class = preds.argmax()

cam.release()

cv2.destroyAllWindows()

KeyboardInterrupt: 

In [31]:
categories = { 
    "chair": [1, 3, 4, 6, 15, 22, 23, 26, 29, 43, 55, 58, 63, 104],
    "table": [8, 13, 14, 19, 59, 61, 80, 83, 90, 96, 103, 113, 126, 128],
    "accessories": [2, 9, 12, 41, 50, 53, 54, 56, 60, 68, 69, 71, 88, 98, 111, 116, 123],
    "kitchen_items": [5, 11, 16, 20, 27, 33, 44, 46, 49, 57, 64, 72, 73, 74, 79, 82, 87, 95, 100, 102, 105, 106, 107, 108, 109, 110, 112, 114, 118, 125, 127],
    "light": [30, 37, 51, 84, 89, 93, 97],
    "bed": [10, 28, 35, 38, 70, 92, 117],
    "couch": [17, 18, 21, 45, 48, 86, 91, 94, 119, 120],
    "electronics": [7, 36, 42, 52, 77, 78, 81, 115, 124],
    "shelves": [24, 25, 31, 32, 34, 40, 47, 62, 65, 66, 67, 75, 85, 122, 121],
    "carpet": [99],
    "other": [39, 76, 101]
}

In [32]:
import functools
    
len(functools.reduce(lambda x, y: x + y, categories.values()))

128

In [33]:
import pandas as pd
import numpy as np
from skimage.io import imread
from skimage.transform import resize
import os

import keras
from keras.models import Model
from keras.layers import Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.preprocessing import image
from keras.applications.inception_v3 import preprocess_input, decode_predictions, InceptionV3
from keras.models import Sequential

import random

/home/mariosk/anaconda2/envs/ipykernel_py3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [34]:
num_classes = 128

model = InceptionV3(weights='imagenet')

intermediate_layer_model = Model(inputs=model.input, outputs=model.layers[311].output)

x = intermediate_layer_model.output
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

transfer_model = Model(inputs=intermediate_layer_model.input, outputs=predictions)

# train last cluster and dense layer
for layer in transfer_model.layers:
    layer.trainable = False

# can train more 
for i in range(311,313):
    transfer_model.layers[i].trainable = True

transfer_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# using checkpoints and early stopping on validation sample to prevent overfitting
# best weight is saved to file_path
checkpoints_filepath="../checkpoints/weights_base.best.hdf5"
checkpoint = ModelCheckpoint(checkpoints_filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early = EarlyStopping(monitor="val_loss", mode="min", patience=2)
callbacks_list = [checkpoint, early] 

In [35]:
transfer_model.load_weights('../checkpoints/weights_base_2.best.hdf5')

In [37]:
from skimage.io import imread
from skimage.transform import resize
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

def load_images(input_dir, size):
    filenames = os.listdir(input_dir)
    
    result = []
    
    for filename in filenames:
        image_name = filename.split('.')[0]
        image_id = image_name.split('_')[1]
        try:
            image = imread(os.path.join(input_dir, filename))
            if (len(image.shape) != 3) or (image.shape[2] != 3):
                raise Exception('Bad image dimensions')
            result.append([image_id, resize(image, size)])
        except Exception as e:
            pass
            print(e)
            print(image_name.split('_')[1])
            
    return result

In [ ]:
val_images = load_images('../data/validation/', (299, 299))

In [40]:
online_images = load_images('../online_images/', (299, 299))

/home/mariosk/anaconda2/envs/ipykernel_py3/lib/python3.6/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


In [41]:
online_images

[['3', array([[[0.33750011, 0.31844768, 0.30211703],
          [0.45622835, 0.43056765, 0.40857276],
          [0.45978382, 0.43412312, 0.41212824],
          ...,
          [0.44461789, 0.42156863, 0.37968762],
          [0.44639563, 0.42156863, 0.38324309],
          [0.33205656, 0.31300413, 0.28578638]],
  
         [[0.35870463, 0.33832297, 0.32085297],
          [0.48633866, 0.45888768, 0.43535827],
          [0.49320155, 0.46575057, 0.44222116],
          ...,
          [0.45584686, 0.4311895 , 0.38638656],
          [0.4669267 , 0.44036758, 0.39936817],
          [0.35053931, 0.33015764, 0.30104098]],
  
         [[0.33851286, 0.3181312 , 0.3006612 ],
          [0.46009346, 0.43264248, 0.40911307],
          [0.46898214, 0.44153116, 0.41800175],
          ...,
          [0.47713378, 0.45247643, 0.40767349],
          [0.50326237, 0.47670325, 0.43570384],
          [0.38275753, 0.36237586, 0.3332592 ]],
  
         ...,
  
         [[0.41149441, 0.39111275, 0.36199608],
         

In [55]:
transfer_model.predict(online_images[0][1].reshape([1, 299, 299, 3])).argmax() + 1

128